In [362]:
import aoc

data = aoc.read("day12.txt")

operation_records, contiguous_damaged = [], []
for line in data.splitlines():
    l, r = line.split()
    operation_records.append(l)
    contiguous_damaged.append(tuple(aoc.to_ints(r.split(","))))

# Part 1

In [352]:
import itertools
import tqdm

In [ ]:
def get_contiguous_from_record(record):
    record = record + "."
    count = 0
    contiguous_damaged = []
    for char in record:
        if char == ".":
            if count > 0:
                contiguous_damaged.append(count)
            count = 0
        elif char == "#":
            count += 1
        else:
            raise RuntimeError(f"Unknown {char=}")
    return contiguous_damaged


def replace_record(record, option):
    record_option = ""
    for char in record:
        if char in ".#":
            record_option += char
        elif char == "?":
            record_option += option.pop(0)
        else:
            raise RuntimeError(f"Unknown {char}")
    return record_option


def n_options_damaged_record(damaged_record, true_contiguous):
    n_unknown = damaged_record.count("?")
    options = itertools.product(".#", repeat=n_unknown)
    n_options = 0
    for option in options:
        record_option = replace_record(damaged_record, list(option))
        contiguous = get_contiguous_from_record(record_option)
        if contiguous == true_contiguous:
            n_options += 1
    return n_options


n_total = 0
for damaged_record, true_contiguous in tqdm.tqdm(
    zip(operation_records, contiguous_damaged), total=len(operation_records)
):

    n_rec = n_options_damaged_record(damaged_record, true_contiguous)
    n_total += n_rec
print(n_total)

# Part 2

In [364]:
import functools

In [375]:
@functools.cache
def get_contiguous_from_record(record):
    record = record + "."
    count = 0
    contiguous_damaged = []
    for char in record:
        if char == ".":
            if count > 0:
                contiguous_damaged.append(count)
            count = 0
        elif char == "#":
            count += 1
        else:
            raise RuntimeError(f"Unknown {char=}")
    return tuple(contiguous_damaged)

In [ ]:
def get_min_max_per_part(part):
    if part.startswith("??") or part == "?":
        n_min = 1
    elif part.startswith("?#"):
        # n_max is later overwritten with the original `len`
        n_min, n_max = get_min_max_per_part(part[1:])
    elif part.startswith("#"):
        try:
            n_min = part.index("?")
        except ValueError:
            n_min = len(part)
    n_max = len(part)
    return n_min, n_max


def first_occurrence_or_len_str(full_str, sub_str):
    try:
        return full_str.index(sub_str)
    except ValueError:
        return len(full_str)


def find_beginning_end_partial_record(record):
    first_definite_broken = first_occurrence_or_len_str(record, "#")
    first_question_mark = first_occurrence_or_len_str(record, "?")
    if first_definite_broken == first_question_mark == len(record):
        return 0, 0
    start_possible_broken = min(first_question_mark, first_definite_broken)

    ending_first_part = first_occurrence_or_len_str(record[first_definite_broken:], ".")
    return start_possible_broken, first_definite_broken + ending_first_part


def find_min_max_broken(record):
    parts = (x for x in record.split(".") if x)
    mins_maxes = [get_min_max_per_part(part) for part in parts]
    n_min = min(x[0] for x in mins_maxes)
    n_max = max(x[1] for x in mins_maxes)
    return n_min, n_max


@functools.cache
def n_options_broken_record(record, true_contiguous):
    if "?" not in record:
        return get_contiguous_from_record(record) == true_contiguous
    if not true_contiguous:
        return "#" not in record
    aiming_for = true_contiguous[0]
    n_options = 0

    for option in ".#":
        new_record = record.replace("?", option, 1)
        beginning_partial, end_partial = find_beginning_end_partial_record(new_record)
        partial_record = new_record[beginning_partial:end_partial]
        if not partial_record:
            continue
        n_min, n_max = find_min_max_broken(partial_record)
        if n_min == n_max == aiming_for:
            first_definite_broken = first_occurrence_or_len_str(new_record, "#")
            first_question_mark = first_occurrence_or_len_str(new_record, "?")
            if first_definite_broken < first_question_mark:
                # It is important to do  + 1. After the exact group
                # (this is exact because of the if), there must always be a ".".
                # We cannot later try whether that's "#"" anyway
                n_options += n_options_broken_record(
                    new_record[beginning_partial + aiming_for + 1 :],
                    true_contiguous[1:],
                )
            elif first_question_mark < first_definite_broken:
                n_options += n_options_broken_record(
                    new_record[first_question_mark:], true_contiguous
                )
            else:
                assert False, "Havent thought this through, not sure this can happen"

        elif aiming_for in range(n_min, n_max + 1):
            n_options += n_options_broken_record(
                new_record[beginning_partial:], true_contiguous
            )
        # This is overexplicit. Technically, this condition could have been skipped
        # But just to show: we failed if aiming for not in range
        else:
            n_options += 0
    return n_options


def run_tests(cases, func):
    for test_input, desired_output in cases:
        aoc.test(func(test_input), desired_output)
    print("All tests OK!")


cases_min_max_part = [
    ("??##", (1, 4)),
    ("##??", (2, 4)),
    ("?##", (2, 3)),
    ("##", (2, 2)),
    ("?", (1, 1)),
]
run_tests(cases_min_max_part, get_min_max_per_part)

cases_partial_record = [
    ("##.?", (0, 2)),
    ("#??", (0, 3)),
    ("##.???", (0, 2)),
    ("???.##", (0, 6)),
    ("..#", (2, 3)),
    (".??..??...?##.", (1, 13)),
    ("...", (0, 0)),
]
run_tests(cases_partial_record, find_beginning_end_partial_record)


cases_min_max_total = [
    ("##", (2, 2)),
    ("#??", (1, 3)),
    ("##", (2, 2)),
    ("???.##", (1, 3)),
    ("??..??...?##", (1, 3)),
]
run_tests(cases_min_max_total, find_min_max_broken)

aoc.test(n_options_broken_record("?", (1,)), 1)
aoc.test(n_options_broken_record(".?", (1,)), 1)
aoc.test(n_options_broken_record("#?", (1,)), 1)

aoc.test(n_options_broken_record("??", (1,)), 2)
aoc.test(n_options_broken_record(".?", (1, 1)), 0)
aoc.test(n_options_broken_record("#?", (1, 1)), 0)

aoc.test(n_options_broken_record("??", (1, 1)), 0)
aoc.test(n_options_broken_record("???", (1, 1)), 1)
aoc.test(n_options_broken_record("???.", (1, 1)), 1)
aoc.test(n_options_broken_record("?.#", (1, 1, 1)), 0)
aoc.test(n_options_broken_record(".??.#", (1, 1, 1)), 0)
aoc.test(n_options_broken_record("##?.#", (1, 1, 1)), 0)
aoc.test(n_options_broken_record("#.#.#", (1, 1, 1)), 1)
aoc.test(n_options_broken_record("#...#", (1, 1, 1)), 0)
aoc.test(n_options_broken_record("#.?.#", (1, 1, 1)), 1)
aoc.test(n_options_broken_record("#??.#", (1, 1, 1)), 1)
aoc.test(n_options_broken_record("???.#", (1, 1, 1)), 1)
aoc.test(n_options_broken_record("???.##", (1, 1, 1)), 0)
aoc.test(n_options_broken_record("???.##", (1, 1, 2)), 1)
aoc.test(n_options_broken_record("???.###", (1, 1, 3)), 1)

aoc.test(n_options_broken_record("###?#?", (6,)), 1)
aoc.test(n_options_broken_record("#?#?#?", (6,)), 1)
aoc.test(n_options_broken_record(".#.#?#?#?", (1, 6)), 1)
aoc.test(n_options_broken_record(".#?#?#?#?", (1, 6)), 1)
aoc.test(n_options_broken_record("?#?#?#?#?", (1, 6)), 1)
aoc.test(n_options_broken_record("###?#?#?#?#?", (3, 1, 6)), 1)
aoc.test(n_options_broken_record(".#.###?#?#?#?#?", (1, 3, 1, 6)), 1)
aoc.test(n_options_broken_record("???.###", (1, 1, 3)), 1)
aoc.test(n_options_broken_record(".??..??...?##.", (1, 1, 3)), 4)
aoc.test(n_options_broken_record("?#?#?#?#?#?#?#?", (1, 3, 1, 6)), 1)
aoc.test(n_options_broken_record("????.#...#...", (4, 1, 1)), 1)

aoc.test(n_options_broken_record("????.######..#####.", (1, 6, 5)), 4)
aoc.test(n_options_broken_record("?###????????", (3, 2, 1)), 10)

In [379]:
unfolded_records = ["?".join([record] * 5) for record in operation_records]
unfolded_contiguous = [contiguous * 5 for contiguous in contiguous_damaged]

In [ ]:
n_total = 0
for damaged_record, true_contiguous in tqdm.tqdm(
    zip(unfolded_records, unfolded_contiguous), total=len(unfolded_contiguous)
):

    n_rec = n_options_broken_record(damaged_record, true_contiguous)
    # print(n_rec)
    n_total += n_rec
print(n_total)